In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error,mean_absolute_error,root_mean_squared_error
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import re
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score
import xarray as xr


In [ ]:
# 加载数据
df=pd.read_csv("prepare_2014-2022.csv",index_col=0)
mask = (df['longitude'] > 119.5) & (df['latitude'] < 26.5)

# 删除这些行
df = df[~mask]
df = df[df['exact_time'] >= 998406]
a12=pd.read_csv("E:\\2023\\weather\\main\\O3finish\\allO3_2013.csv",index_col=0)
#a12=a12[a12['time'] == 990558]
a12= a12.reset_index(drop=True)
print(a12)

In [ ]:
df = df[~df.isin([ -9999.0]).any(axis=1)]


In [ ]:
def filter_by_year(df, year,ww):
    # 基准时间
    base_time = pd.Timestamp('1900-01-01 00:00:00')
    
    # 将 'exact_time' 转换为实际的日期时间
    df['datetime'] = base_time + pd.to_timedelta(df['exact_time'], unit='h')
    
    # 筛选出指定年份的数据
    if ww:
        filtered_df = df[df['datetime'].dt.year == year]
    else:
        filtered_df = df[df['datetime'].dt.year != year]
    
    # 删除辅助的 'datetime' 列，保持原表格格式
    filtered_df = filtered_df.drop(columns=['datetime'])
    
    return filtered_df


In [ ]:
df["num"]=(df["longitude"]-100)*2+47*(df["latitude"]-20)
yearsd=2019
new_names = {col: re.sub(r'[^A-Za-z0-9_]+', '', col) for col in df.columns}
new_n_list = list(new_names.values())
# [LightGBM] Feature appears more than one time.
new_names = {col: f'{new_col}_{i}' if new_col in new_n_list[:i] else new_col for i, (col, new_col) in enumerate(new_names.items())}
gdf=df.copy()

gdf = gdf.rename(columns=new_names)
y = gdf.iloc[:, [0,3]]

gdf.drop(df.columns[3], axis=1, inplace=True)
gdf.drop(['longitude', 'latitude'], axis=1, inplace=True)
gdf.drop(['num'], axis=1, inplace=True)
X = gdf
X_less=X.iloc[:,:]
X_less=X_less.iloc[:,:]

#X_train, X_test, y_train, y_test = train_test_split(X_less.iloc[:,1:], y.iloc[:,1], test_size=0.2,random_state=42)

X_test=filter_by_year(X_less,yearsd,True)
X_test=X_test.iloc[:,1:]
X_train = filter_by_year(X_less,yearsd,False)
X_train = X_train.iloc[:,1:]
y_test=filter_by_year(y,yearsd,True)
y_test=y_test.iloc[:,1]
y_train = filter_by_year(y,yearsd,False)
y_train=y_train.iloc[:,1]
print(y_test)
# 转换为Dataset数据格式
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)




In [ ]:
# 参数
params = {
    'task': 'train',
    'boosting_type': 'gbdt',  # 设置提升类型
    'objective': 'regression',  # 回归任务
    'metric': {'l2', 'auc'},  # 评估函数
    'num_leaves': 20000,  # 叶子节点数
    'learning_rate': 0.1,  # 学习速率
    'feature_fraction': 0.95,  # 建树的特征选择比例
    'bagging_fraction': 0.95,  # 建树的样本采样比例
    'bagging_freq': 2,  # k 意味着每 k 次迭代执行bagging
    'verbose': 1 , # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
    'device': 'gpu'
}

# 模型训练
gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval)

# 模型保存
gbm.save_model('model.txt')

# 模型加载
gbm = lgb.Booster(model_file='model.txt')

# 模型预测
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)



# 模型评估
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)
r2_test = r2_score(y_test, y_pred)
print(f"R^2 for test set: {r2_test:.4f}")

all_elements = list(range(len(y_test)))
    
# 随机打乱列表的顺序
random.shuffle(all_elements)
y_test=np.array(y_test)
y_train=np.array(y_train)
# 选取前100个元素作为索引
selected_indices = all_elements[:1000]
print("drawing")
plt.figure(figsize=(8, 6))
sns.kdeplot(x=y_test[selected_indices], y=y_pred[selected_indices], fill=True, cmap='viridis', cbar=True, label="Test Set")
plt.xlabel("True Values")
plt.ylabel("Predicted Values")
plt.title("with_space_and_time_2013")
plt.plot([20, 200], [20, 200], color='gray', linestyle='--', label="x=y")
plt.legend()
plt.show()

In [ ]:

# 模型预测
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)


# 模型评估
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)
r2_test = r2_score(y_test, y_pred)
print(f"R^2 for test set: {r2_test:.4f}")

all_elements = list(range(len(y_test)))
    
# 随机打乱列表的顺序
random.shuffle(all_elements)
y_test=np.array(y_test)
y_train=np.array(y_train)
# 选取前100个元素作为索引
selected_indices = all_elements[:1000]
print("drawing")
plt.figure(figsize=(8, 6))
sns.kdeplot(x=y_test[selected_indices], y=y_pred[selected_indices], fill=True, cmap='viridis', cbar=True, label="Test Set")
plt.xlabel("True Values")
plt.ylabel("Predicted Values")
plt.title("with_space_and_time_2013")
plt.plot([20, 200], [20, 200], color='gray', linestyle='--', label="x=y")
plt.legend()
plt.show()

In [ ]:
X_test=filter_by_year(X_less,yearsd,True)
X_train = filter_by_year(X_less,yearsd,False)
y_test=filter_by_year(y,yearsd,True)
y_test=y_test.iloc[:,1]
y_train = filter_by_year(y,yearsd,False)
y_train=y_train.iloc[:,1]


#X_train, X_test, y_train, y_test = train_test_split(X_less.iloc[:,:], y.iloc[:,1], test_size=0.2,random_state=42)


y_more=X_test.iloc[:,[0,1,2,3]]
y_more = y_more.reset_index(drop=True) 


y_pred_frame = pd.DataFrame({'O3_pred': y_pred})
y_pred_frame=y_pred_frame.reset_index(drop=True)
y_test_frame=  pd.DataFrame({"O3_true":y_test})
y_test_frame=y_test_frame.reset_index(drop=True)
result_df = pd.concat([y_more, y_pred_frame], axis=1)
result_df = result_df.drop(result_df.columns[3], axis=1)
original_df=pd.concat([y_more, y_test_frame], axis=1)
original_df = original_df.drop(original_df.columns[3], axis=1)


In [ ]:
print(original_df)
result_df.to_csv("temp.csv")

In [ ]:

# 计算相关系数、RMSE和MAE
r2 = r2_score(original_df["O3_true"], result_df["O3_pred"])
rmse = mean_squared_error(original_df["O3_true"], result_df["O3_pred"], squared=False)  # squared=False表示计算RMSE
mae = mean_absolute_error(original_df["O3_true"], result_df["O3_pred"])
x_range = np.arange(20, 43.5, 0.5)
y_range = np.arange(100, 123.5, 0.5)


# 创建网格数据
xx, yy = np.meshgrid(x_range, y_range)
result_temp= np.column_stack((xx.ravel(), yy.ravel()))
np.set_printoptions(precision=1,suppress=True)

aa=pd.DataFrame(result_temp,columns=["latitude","longitude"])
aa["rmse"]=0.0
aa["mae"]=0.0
aa["r2"]=0.0
sorted_aa = aa.sort_values(by=["latitude","longitude"])
print(aa)
for i in range(0,len(aa)):
    lon1=aa.iloc[i,1];lat1=aa.iloc[i,0]
    # 筛选DataFrame
    filtered_ori = original_df[(original_df['longitude'] == lon1) & (original_df['latitude'] == lat1)]

    # 检查筛选后的DataFrame是否为空
    if filtered_ori.empty:
        shabizhouxianrong=1+1
    else:
        filtered_result = result_df[(result_df['longitude'] == lon1) & (result_df['latitude'] == lat1)]

        aa.iloc[i,4]=r2_score(filtered_ori["O3_true"], filtered_result["O3_pred"])
        aa.iloc[i,2]=root_mean_squared_error(filtered_ori["O3_true"], filtered_result["O3_pred"])
        aa.iloc[i,3]=mean_absolute_error(filtered_ori["O3_true"], filtered_result["O3_pred"])
aa.to_csv(".\\output\\"+str(yearsd)+"_pointwise_r2_rmse_mae.csv")
        

In [ ]:
import pandas as pd
import xarray as xr


# Assuming 'exact_time' is in hours since '1900-01-01 00:00:00.0'
base_time = pd.Timestamp('1900-01-01 00:00:00')

# First, ensure 'exact_time' is treated as numeric (hours since base_time)
result_df['exact_time'] = pd.to_numeric(result_df['exact_time'])

# Calculate the actual datetime from hours since base_time
result_df['exact_time'] = base_time + pd.to_timedelta(result_df['exact_time'], unit='h')

# Set up the dimensions and variable for xarray
ds = result_df.set_index(['exact_time', 'latitude', 'longitude']).to_xarray()

# Save the Dataset as a NetCDF file
output_path = ".//output//"+"result_"+str(yearsd)+".nc"
ds.to_netcdf(output_path)




In [ ]:
# Assuming 'exact_time' is in hours since '1900-01-01 00:00:00.0'
base_time = pd.Timestamp('1900-01-01 00:00:00')

# First, ensure 'exact_time' is treated as numeric (hours since base_time)
original_df['exact_time'] = pd.to_numeric(original_df['exact_time'])

# Calculate the actual datetime from hours since base_time
original_df['exact_time'] = base_time + pd.to_timedelta(original_df['exact_time'], unit='h')

# Set up the dimensions and variable for xarray
ds = original_df.set_index(['exact_time', 'latitude', 'longitude']).to_xarray()

# Save the Dataset as a NetCDF file
output_path = ".//output//"+"original_"+str(yearsd)+".nc"
ds.to_netcdf(output_path)

In [ ]:
import netCDF4 as nc

# 打开.nc文件
dataset = nc.Dataset(output_path)

# 查看文件中的变量
variables = dataset.variables
for var_name in variables:
    var = dataset.variables[var_name]
    print(f'变量名: {var_name}')
    print(f'维度: {var.dimensions}')
    print(f'形状: {var.shape}')
    print(f'属性: {var.ncattrs()}')
    for attr_name in var.ncattrs():
        print(f'{attr_name}: {getattr(var, attr_name)}')

variable_names = list(dataset.variables.keys())

# 打印变量名列表
print(variable_names)

# 关闭文件
dataset.close()

In [ ]:
output_path = ".//output//"+"result_"+str(yearsd)+".nc"
# Load the NetCDF file
file_path = output_path # Replace with the path to your NetCDF file
ds = xr.open_dataset(file_path)

# Check the available 'exact_time' values
print(ds['exact_time'])

# Select a specific date (e.g., the first date in the dataset)
selected_date = ds['exact_time'].isel(exact_time=60)  # Change index to select the desired date

# Extract the 'O3_true' data for the selected date
o3_data = ds['O3_pred'].sel(exact_time=selected_date)

# Create a plot of 'O3_true' for the selected date
plt.figure(figsize=(10, 6))
o3_data.plot(x='longitude', y='latitude', cmap='viridis')  # Use x as longitude and y as latitude
plt.title(f'O3_true on {selected_date.values}')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.colorbar(label='O3_pred')
plt.show()

In [ ]:
output_path = ".//output//"+"original_"+str(yearsd)+".nc"
# Load the NetCDF file
file_path = output_path # Replace with the path to your NetCDF file
ds = xr.open_dataset(file_path)

# Check the available 'exact_time' values
print(ds['exact_time'])

# Select a specific date (e.g., the first date in the dataset)
selected_date = ds['exact_time'].isel(exact_time=60)  # Change index to select the desired date

# Extract the 'O3_true' data for the selected date
o3_data = ds['O3_true'].sel(exact_time=selected_date)

# Create a plot of 'O3_true' for the selected date
plt.figure(figsize=(10, 6))
o3_data.plot(x='longitude', y='latitude', cmap='viridis')  # Use x as longitude and y as latitude
plt.title(f'O3_true on {selected_date.values}')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.colorbar(label='O3_pred')
plt.show()

In [ ]:

# 指定时间范围
start_time = 0
end_time = 2160*4

# 筛选符合时间范围的数据
filtered_df1 = result_df
print(result_df)

# 按["longitude", "latitude"]组合计算平均值
grouped_df1 = filtered_df1.groupby(['longitude', 'latitude'])['O3_pred'].mean().reset_index()


filtered_df2 = original_df

# 按["longitude", "latitude"]组合计算平均值
grouped_df2 = filtered_df2.groupby(['longitude', 'latitude'])['O3_true'].mean().reset_index()


# 指定时间范围
temp_data1_df=original_df.copy()
temp_data1_df["O3_true"]=(result_df["O3_pred"]-original_df["O3_true"])/1
#original_df["O3_true"]=(result_df["O3_pred"]-original_df["O3_true"])/2
# 筛选符合时间范围的数据
# filtered_df2 = original_df[(original_df['time'] >= start_time) & (original_df['time'] <= end_time)]

# # 按["longitude", "latitude"]组合计算平均值
# grouped_df2 = filtered_df2.groupby(['longitude', 'latitude'])['O3_true'].mean().reset_index()
filtered_df3 = temp_data1_df

# 按["longitude", "latitude"]组合计算平均值
grouped_df3 = filtered_df3.groupby(['longitude', 'latitude'])['O3_true'].mean().reset_index()
# 绘制散点图
plt.figure(figsize=(10, 6))
sns.scatterplot(x='longitude', y='latitude', hue='O3_pred', size='O3_pred', data=grouped_df1, palette='coolwarm', legend='brief')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Q3 2017 mean error(pred-original)(without emmision)')
plt.show()
plt.figure(figsize=(10, 6))
sns.scatterplot(x='longitude', y='latitude', hue='O3_true', size='O3_true', data=grouped_df2, palette='coolwarm', legend='brief')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Q3 2017 mean error(pred-original)(without emmision)')
plt.show()